In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets tensorflow tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
import os
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import TFAutoModel, AutoTokenizer

In [4]:
model_name = "bert-base-uncased"
model = TFAutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
dataset = load_dataset('dair-ai/emotion', 'split')

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [7]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True,max_length=55)

In [8]:
tokenized_dataset = dataset.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [9]:
tokenized_dataset.set_format('tf', columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

In [10]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [11]:
BATCH_SIZE = 64

In [12]:
example = tokenized_dataset['train'][1]
data = list(example.values())

In [13]:
print("Input IDs:", data[1])

Input IDs: tf.Tensor(
[  101  1045  2064  2175  2013  3110  2061 20625  2000  2061  9636 17772
  2074  2013  2108  2105  2619  2040 14977  1998  2003  8300   102     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0], shape=(55,), dtype=int64)


In [14]:
print("Label:", data[0])

Label: tf.Tensor(0, shape=(), dtype=int64)


In [15]:
print("Attention Mask:", data[3])

Attention Mask: tf.Tensor(
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(55,), dtype=int64)


In [16]:
print("token_type_ids :", data[2])

token_type_ids : tf.Tensor(
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(55,), dtype=int64)


In [17]:
def order(inp):
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[3],
        'token_type_ids': data[2]
    }, data[0]

In [18]:
train_dataset = tf.data.Dataset.from_tensor_slices(tokenized_dataset['train'][:])
train_dataset = train_dataset.shuffle(1000).batch(BATCH_SIZE).map(order, num_parallel_calls=tf.data.AUTOTUNE)

validation_dataset = tf.data.Dataset.from_tensor_slices(tokenized_dataset['validation'][:])
validation_dataset = validation_dataset.batch(BATCH_SIZE).map(order, num_parallel_calls=tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices(tokenized_dataset['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE).map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [19]:
inp, out = next(iter(train_dataset))
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(64, 55), dtype=int64, numpy=
array([[ 101, 1045, 7039, ...,    0,    0,    0],
       [ 101, 1045, 2514, ...,    0,    0,    0],
       [ 101, 1045, 2031, ...,    0,    0,    0],
       ...,
       [ 101, 1045, 2145, ...,    0,    0,    0],
       [ 101, 1045, 2938, ...,    0,    0,    0],
       [ 101, 1045, 2001, ...,    0,    0,    0]])>, 'attention_mask': <tf.Tensor: shape=(64, 55), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(64, 55), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>} 

 tf.Tensor(
[1 1 3 0 3 4 1 3 0 1 3 3 0 2 5 1 1 0 0 1 1 0 0 3 3 3 1 0 3 2 3 

In [20]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes, dropout_rate=0.3, l2_reg=0.01):
        super().__init__()
        self.bert = bert_model
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(l2_reg))

    def call(self, inputs):
        x = self.bert(inputs)[1]
        x = self.dropout(x)
        return self.fc(x)

In [21]:
classifier = BERTForClassification(model, num_classes=6, dropout_rate=0.3, l2_reg=0.005)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [22]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

In [23]:
history = classifier.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=50,
    verbose=1,
    callbacks=[early_stopping]
)

Epoch 1/50
250/250 [==============================] - 219s 704ms/step - loss: 1.1594 - accuracy: 0.6104 - val_loss: 0.5396 - val_accuracy: 0.8335
Epoch 2/50
250/250 [==============================] - 181s 724ms/step - loss: 0.3841 - accuracy: 0.8878 - val_loss: 0.2710 - val_accuracy: 0.9195
Epoch 3/50
250/250 [==============================] - 181s 723ms/step - loss: 0.2282 - accuracy: 0.9341 - val_loss: 0.2262 - val_accuracy: 0.9245
Epoch 4/50
250/250 [==============================] - 181s 722ms/step - loss: 0.1834 - accuracy: 0.9457 - val_loss: 0.2222 - val_accuracy: 0.9345
Epoch 5/50
250/250 [==============================] - 180s 721ms/step - loss: 0.1628 - accuracy: 0.9536 - val_loss: 0.2285 - val_accuracy: 0.9340
Epoch 6/50
250/250 [==============================] - 180s 722ms/step - loss: 0.1482 - accuracy: 0.9595 - val_loss: 0.2168 - val_accuracy: 0.9340
Epoch 7/50
250/250 [==============================] - 180s 722ms/step - loss: 0.1362 - accuracy: 0.9632 - val_loss: 0.2359 -

In [24]:
evaluation_results = classifier.evaluate(test_dataset)

32/32 [==============================] - 8s 234ms/step - loss: 0.2366 - accuracy: 0.9220


In [25]:
# base_save_path = os.path.expanduser('~/my_models_third/')
base_save_path = '/content/drive/MyDrive/Projects/youtubecommentsentimetanalysis/model'

In [26]:
model_save_path_tf = os.path.join(base_save_path, 'saved_model_second')
classifier.save(model_save_path_tf, save_format='tf')

In [27]:
tokenizer_save_path = os.path.join(base_save_path, 'saved_tokenizer')
tokenizer.save_pretrained(tokenizer_save_path)

('/content/drive/MyDrive/Projects/youtubecommentsentimetanalysis/model/saved_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/Projects/youtubecommentsentimetanalysis/model/saved_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/Projects/youtubecommentsentimetanalysis/model/saved_tokenizer/vocab.txt',
 '/content/drive/MyDrive/Projects/youtubecommentsentimetanalysis/model/saved_tokenizer/added_tokens.json',
 '/content/drive/MyDrive/Projects/youtubecommentsentimetanalysis/model/saved_tokenizer/tokenizer.json')

In [28]:
results_save_path = os.path.join(base_save_path, 'evaluation_results.txt')
with open(results_save_path, 'w') as f:
    f.write(f"Loss: {evaluation_results[0]}\n")
    f.write(f"Accuracy: {evaluation_results[1]}\n")

In [29]:
model_save_path = os.path.join(base_save_path, 'saved_model')
classifier.save(model_save_path)

In [30]:
!pip freeze > /content/drive/MyDrive/Projects/youtubecommentsentimetanalysis/model/model-requirements.txt